In [1]:
pip install yfinance


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install yfinance



SyntaxError: invalid syntax (2977875142.py, line 1)

In [ ]:
import yfinance as yf
import pandas as pd

def download_and_resample(symbol, start_date, end_date, interval):
    try:
        print(f"\n📥 Downloading {symbol} data from {start_date} to {end_date} ({interval})...")

        # Download OHLCV data
        df = yf.download(symbol, start=start_date, end=end_date, interval=interval, progress=False)

        if df.empty:
            print("❌ No data found. Check your symbol, date range, or interval.")
            return

        # Reset index to separate date & time
        df.reset_index(inplace=True)

        # Detect correct datetime column name
        datetime_col = 'Datetime' if 'Datetime' in df.columns else 'Date'
        df['date'] = pd.to_datetime(df[datetime_col]).dt.date
        df['time'] = pd.to_datetime(df[datetime_col]).dt.time

        # Select and rename columns
        df = df[['date', 'time', 'Open', 'High', 'Low', 'Close', 'Volume']]
        df.columns = ['date', 'time', 'open', 'high', 'low', 'close', 'volume']

        # Drop missing values
        df.dropna(inplace=True)

        # Combine date and time for resampling
        df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))
        df.set_index('datetime', inplace=True)

        # ✅ Map Yahoo interval → Pandas resample frequency
        interval_map = {
            '1m': '1T', '2m': '2T', '5m': '5T', '15m': '15T', '30m': '30T',
            '60m': '60T', '90m': '90T', '1h': '1H',
            '1d': '1D', '5d': '5D', '1wk': '1W', '1mo': '1M', '3mo': '3M'
        }

        pandas_interval = interval_map.get(interval, '1D')

        # Resample
        resampled = df.resample(pandas_interval).agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).dropna()

        # Extract date/time
        resampled['date'] = resampled.index.date
        resampled['time'] = resampled.index.time
        resampled = resampled[['date', 'time', 'open', 'high', 'low', 'close', 'volume']]

        # Save to CSV
        filename = f"{symbol}_{interval}.csv"
        resampled.to_csv(filename, index=False)

        print(f"✅ File saved: {filename}")
        print("\n📊 Sample Data:\n", resampled.head())

    except Exception as e:
        print(f"⚠️ Error: {e}")


# ========== MAIN PROGRAM ==========
if __name__ == "__main__":
    symbol = input("Enter Stock Symbol (e.g., SBIN.NS, RELIANCE.NS): ").strip()
    start_date = input("Enter Start Date (YYYY-MM-DD): ").strip()
    end_date = input("Enter End Date (YYYY-MM-DD): ").strip()
    interval = input("Enter Timeframe (e.g., 5m, 15m, 1h, 1d): ").strip()

    valid_intervals = ['1m','2m','5m','15m','30m','60m','90m','1h','1d','5d','1wk','1mo','3mo']
    if interval not in valid_intervals:
        print("❌ Invalid interval! Please choose one of:", valid_intervals)
    else:
        download_and_resample(symbol, start_date, end_date, interval)

        

In [ ]:
from flask import Flask, request, jsonify
import yfinance as yf
import pandas as pd

app = Flask(__name__)

def download_and_resample(symbol, start_date, end_date, interval):
    try:
        df = yf.download(symbol, start=start_date, end=end_date, interval=interval, progress=False)

        if df.empty:
            return None, "No data found. Check symbol (.NS for NSE), date range, or interval."

        df.reset_index(inplace=True)

        # Flatten MultiIndex columns
        df.columns = ['_'.join([str(c) for c in col if c]).strip().lower()
                      if isinstance(col, tuple) else str(col).lower()
                      for col in df.columns]

        # Detect datetime column
        datetime_col = next((c for c in df.columns if c.startswith("date") or c.startswith("datetime")), None)

        if not datetime_col:
            return None, f"No valid datetime column found. Columns present: {df.columns.tolist()}"

        df['datetime'] = pd.to_datetime(df[datetime_col])

        # Detect OHLCV columns dynamically
        col_map = {
            'open': next((c for c in df.columns if c.startswith("open")), None),
            'high': next((c for c in df.columns if c.startswith("high")), None),
            'low': next((c for c in df.columns if c.startswith("low")), None),
            'close': next((c for c in df.columns if c.startswith("close")), None),
            'volume': next((c for c in df.columns if c.startswith("vol")), None)
        }

        if None in col_map.values():
            return None, f"Missing required OHLCV columns. Found: {df.columns.tolist()}"

        df = df[['datetime', col_map['open'], col_map['high'], col_map['low'], col_map['close'], col_map['volume']]]
        df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']

        df.set_index('datetime', inplace=True)

        # Interval map
        interval_map = {
            '1m': '1T','2m': '2T','5m': '5T','15m': '15T','30m': '30T',
            '60m': '1H','90m':'90T','1h':'1H','1d':'1D','5d':'5D',
            '1wk':'1W','1mo':'1M','3mo':'3M'
        }
        pandas_interval = interval_map.get(interval)
        if pandas_interval is None:
            return None, f"Invalid interval: {interval}"

        resampled = df.resample(pandas_interval).agg({
            'open': 'first', 'high': 'max', 'low': 'min',
            'close': 'last', 'volume': 'sum'
        }).dropna()

        resampled = resampled.reset_index()

        # FIX JSON Serialization
        resampled['date'] = resampled['datetime'].dt.date.astype(str)
        resampled['time'] = resampled['datetime'].dt.time.astype(str)

        return resampled, None

    except Exception as e:
        return None, str(e)


@app.route("/download", methods=["POST"])
def api_download():
    try:
        data = request.get_json()
        symbol = data.get("symbol")
        start_date = data.get("start_date")
        end_date = data.get("end_date")
        interval = data.get("interval")

        if not all([symbol, start_date, end_date, interval]):
            return jsonify({"error": "Required JSON: symbol, start_date, end_date, interval"}), 400

        df, error = download_and_resample(symbol, start_date, end_date, interval)
        if error:
            return jsonify({"error": error}), 400

        return jsonify(df.to_dict(orient="records"))

    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route("/", methods=["GET"])
def home():
    return jsonify({"message": "Stock OHLCV Resample API is running!"})


if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5003)


In [ ]:
from flask import Flask, request, jsonify
import yfinance as yf
import pandas as pd

app = Flask(__name__)

def download_and_resample(symbol, start_date, end_date, interval):
    try:
        df = yf.download(symbol, start=start_date, end=end_date, interval=interval, progress=False)

        if df.empty:
            return {"error": "No data found. Check your symbol, date range, or interval."}

        df.reset_index(inplace=True)

        datetime_col = 'Datetime' if 'Datetime' in df.columns else 'Date'
        df['date'] = pd.to_datetime(df[datetime_col]).dt.date
        df['time'] = pd.to_datetime(df[datetime_col]).dt.time

        df = df[['date', 'time', 'Open', 'High', 'Low', 'Close', 'Volume']]
        df.columns = ['date', 'time', 'open', 'high', 'low', 'close', 'volume']

        df.dropna(inplace=True)

        df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))
        df.set_index('datetime', inplace=True)

        interval_map = {
            '1m': '1T', '2m': '2T', '5m': '5T', '15m': '15T', '30m': '30T',
            '60m': '60T', '90m': '90T', '1h': '1H',
            '1d': '1D', '5d': '5D', '1wk': '1W', '1mo': '1M', '3mo': '3M'
        }

        pandas_interval = interval_map.get(interval, '1D')

        resampled = df.resample(pandas_interval).agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).dropna()

        resampled['date'] = resampled.index.date.astype(str)
        resampled['time'] = resampled.index.time.astype(str)

        resampled = resampled[['date', 'time', 'open', 'high', 'low', 'close', 'volume']]

        return resampled.to_dict(orient="records")

    except Exception as e:
        return {"error": str(e)}


# ---------------- GET API ----------------
@app.route("/api/data", methods=["GET"])
def get_data():
    symbol = request.args.get("symbol")
    start = request.args.get("start")
    end = request.args.get("end")
    interval = request.args.get("interval")

    if not all([symbol, start, end, interval]):
        return jsonify({"error": "Missing required parameters"}), 400

    data = download_and_resample(symbol, start, end, interval)
    return jsonify(data)


if __name__ == "__main__":
    app.run(debug=True, use_reloader=False, port=5005)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5005/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Nov/2025 10:41:18] "GET / HTTP/1.1" 404 -
C:\Users\user\AppData\Local\Temp\ipykernel_3164\163301998.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date, interval=interval, progress=False)

1 Failed download:
['SBIN.NS']: YFPricesMissingError('possibly delisted; no price data found  (5m 2024-01-01 -> 2024-01-05) (Yahoo error = "5m data not available for startTime=1704047400 and endTime=1704393000. The requested range must be within the last 60 days.")')
127.0.0.1 - - [19/Nov/2025 10:41:34] "GET /api/data?symbol=SBIN.NS&start=2024-01-01&end=2024-01-05&interval=5m HTTP/1.1" 200 -
C:\Users\user\AppData\Local\Temp\ipykernel_3164\163301998.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, end=end_date, interval=interval, progress=False)
127.0.0.1 - 